# TF-IDF Exercise

You have the following (tokenized) document collection:

|id | words|
|---|------|
| 1 | like, like, fruit, fly, fly|
| 2 | bee, wasp, like|
| 3 | fruit, fly|
| 4 | bee, wasp, fruit|
| 5| fruit, fruit, fruit, fly|

1. Determine the Document Term Matrix.

2. Calculate all the tf-idf values.

Use max-normalization, such that the most frequent term of every document has relative term frequency 1.
(Equivalent to k-normalization from the lecture with $K=0$). 

Use the natural logarithm for calculation.

### Solution

#### Document Term Matrix:

| term |  D1| D2|D3|D4|D5| IDF|
|------|----|---|--|--|--|---|
| bee| 0|1|0|1|0|
| wasp|0|1|0|1|0| 
| like|2|1|0|0|0| 
| fruit|1|0|1|1|3|
| fly|2|0|1|0|1|


#### Tf, idf Values:

| term |  D1| D2|D3|D4|D5| IDF|
|------|----|---|--|--|--|---|
| bee| 0|1|0|1|0|0.916|
| wasp|0|1|0|1|0|0.916|
| like|1|1|0|0|0| 0.916|
| fruit|0.5|0|1|1|1|0.223|
| fly|1|0|1|0|0.333|0.511|

$N = 5$

$ {idf}_{bee} = log(\frac{5}{2}) = 0.916 $

$ {tf}_{fruit @ D1} = \frac{1}{2}$

$ {tf}_{fly @ D5} = \frac{1}{3}$ (D5: 3x _fruit_, 1x _fly_)


#### Tf-idf Values:

| term |  D1| D2|D3|D4|D5|
|------|----|---|--|--|--|
| bee  | 0  |0.916|0|0.916|0|
| wasp |0   |0.916|0|0.916|0|
| like |0.916   |0.916 |0|0|0|
| fruit|0.112   |0|0.223|0.223|0.223|
| fly  |0.511|0|0.511|0|0.170|



# A2 - Application Retrieval

We can use the tf-idf table to determine the similarity of two documents. Each document can be represented as the vector of its tf-idf weighted words. The cosine of the two vectors is often used to calculate a similarity measure.

When for example a user searches for "fruit fly", you can interpret the query as a new document and translate the query into a vector corresponding to your vocabulary. Calculate the cosine similarity with every document in your corpus and  return the best match.

$$ cos(\vec{q},\vec{d}) = \frac{\vec{q}\cdot\vec{d}}{\lVert\vec{q}\rVert\cdot\lVert\vec{d}\rVert}$$


Which document would the queries $Q_1=[fruit, fly]$ and $Q_2=[bee, fly]$ return?

To generate a query vector, create a vector that has the same dimension as the vocabulary and 1 if a word occurs in the query and zero everywhere else. (One-Hot Encoding)

What are limitations and problems 
of this approach?


### Solution

$\vec{Q} = (0,0,0,1,1)^T$

Similarities :

| D1| D2| D3 | D4 | D5|
|---|---|----|----|---|
| 0.417| 0. |0.931| 0.159| **0.991**|

Document D5 would be returned


$\vec{Q} = (1,0,0,0,1)^T$

Similarities :

| D1| D2| D3 | D4 | D5|
|---|---|----|----|---|
|0.343| 0.408|**0.648**| 0.487| 0.429|

Document D3 would be returned

In [1]:
# Example Code numpy:
import numpy as np
q1 = np.array([0,0,0,1,1])
q2 = np.array([1,0,0,0,1])
d = np.array([[0,0,0.916,0.112,0.511], # D1
              [0.916,0.916,0.916,0,0], # D2
              [0,0,0,0.223,0.511],
              [0.916,0.916,0,0.223,0],
              [0,0,0,0.223,0.170]]) # D5

# @-operator = matmul (Matrix Multiplication)
print(f"Similarities for Q1: {d@q1 / (np.linalg.norm(q1)*np.linalg.norm(d, axis=1))}")
print(f"Similarities for Q2: {d@q2 / (np.linalg.norm(q1)*np.linalg.norm(d, axis=1))}")

Similarities for Q1: [0.41761867 0.         0.93090556 0.11996042 0.99102857]
Similarities for Q2: [0.34254116 0.40824829 0.64808276 0.49275222 0.4286892 ]


## Problems

- OOV, words can occurr in the query but not the corpus (Out-of-vocabulary)
- Memory when scaling to the vocabulary of large corpora (technical problem)
- No disambiguation
- Stemming, wordforms?

### Computation Notes

In [2]:
#q1   = [0, 0,     0,     1,     1]
#d[0] = [0, 0, 0.916, 0.112, 0.511] # D1
r = sum([ai * bi for ai,bi in zip(q1, d[0])])  # = d[0]@q1
print(r)

print(d@q1) # = np.dot(d, q1); all docs in d with q1
print(np.linalg.norm(q1)) # norm of query
print(np.linalg.norm(d, axis=1)) # norm of docs
print(np.linalg.norm(q1)*np.linalg.norm(d, axis=1))

0.623
[0.623 0.    0.734 0.223 0.393]
1.4142135623730951
[1.05485591 1.58655854 0.55753924 1.31447366 0.28040863]
[1.49179154 2.2437326  0.78847955 1.85894648 0.39655769]
